# Neural Network approach

## Getting the dataset  ready

In [2]:
import pandas as pd
my_csv = pd.read_csv(r"D:\Programming shit\Python\MachineLearningDatasets\creditcard.csv")

In [4]:
import numpy as np
my_csv['split'] = np.random.randn(my_csv.shape[0], 1)

In [83]:
selection = np.random.rand(len(my_csv)) <= 0.85

In [84]:
train_csv = my_csv[selection]
test_csv = my_csv[~selection]

In [85]:
trainreader = csv.reader(train_csv.drop(columns = ['Class','split']), delimiter='t')
testreader = csv.reader(test_csv.drop(columns = ['Class','split']), delimiter='t')
transaction_data_train = []
transaction_data_test = []
for i, line in enumerate(trainreader):
    transaction_data_train.append(line)
for i, line in enumerate(testreader): 
    transaction_data_test.append(line)

In [86]:
transaction_data_train = train_csv.drop(columns = ['Class','split'])
transaction_data_test = test_csv.drop(columns = ['Class','split'])

In [87]:
transaction_labels_train = list(train_csv['Class'])
transaction_labels_test = list(test_csv['Class'])

In [88]:
import torch

class CardTransactionsDataset(torch.utils.data.Dataset):
    def __init__(self,labels,transactions_csv):
        self.labels = labels
        self.data = transactions_csv
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return self.data[idx],self.labels[idx]
        

In [93]:
from torchvision.transforms.functional import to_tensor

def my_collate(data):
    processed_data = []
    for line in data:
        line = to_tensor(line)
        line = line.unsqueeze(0)
        processed_data.append(line)
        newdata = torch.cat(processed_data,dim = 0)
        return newdata

In [94]:
training_data = CardTransactionsDataset(transaction_labels_train,transaction_data_train)
testing_data = CardTransactionsDataset(transaction_labels_test,transaction_data_test)

In [95]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=1000, shuffle=True,collate_fn=my_collate)
test_dataloader = DataLoader(testing_data, batch_size=1000, shuffle=True,collate_fn=my_collate)

## Defining the network

In [ ]:
import torch.nn as nn

class CardNet(nn.Module):
    def __init__(self):
        super().__init__()

## Actual training

# KNN approach

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [3]:
my_csv2 = pd.read_csv(r"D:\Programming shit\Python\MachineLearningDatasets\creditcard.csv")

In [45]:
transactions = my_csv2.iloc[:,0:30]
labels = my_csv2.iloc[:,30]
training_transactions, testing_transactions,training_labels,testing_labels = train_test_split(transactions,labels,test_size = .15)

In [46]:
transactions_scaling = StandardScaler()
training_transactions = transactions_scaling.fit_transform(training_transactions)
testing_transactions = transactions_scaling.fit_transform(testing_transactions)

In [47]:
from math import sqrt,trunc
print(trunc(sqrt(training_transactions.shape[0]))-1)

491


In [50]:
knn = KNeighborsClassifier(n_neighbors= 491,p = 2,metric = 'euclidean')

In [51]:
knn.fit(training_transactions,training_labels)

KNeighborsClassifier(metric='euclidean', n_neighbors=491)

In [56]:
predicted_labels = knn.predict(testing_transactions)

In [57]:
confusion_matrix = confusion_matrix(testing_labels,predicted_labels)

In [58]:
print(confusion_matrix)

[[42645     8]
 [   47    22]]


In [61]:
print("The model had an f1 score of "+ str(f1_score(testing_labels,predicted_labels))+ " and an accuracy score of "+str(accuracy_score(testing_labels,predicted_labels)))

The model had an f1 score of 0.4444444444444444 and an accuracy score of 0.9987126070876832
